In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('C:\\Users\\Irina\\Documents\\Материалы PYDA\\Stats\\Visualization\\video_games_sales.csv')
data.head()

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


 ### Как критики относятся к спортивным играм?

#### Гипотезы

H0 = оценка критиков по спортивным играм <= 5
H1 = оценка критиков по спортивным играм > 5

In [3]:
sports_filterd = data[data['Genre'] == 'Sports']
sports_filterd[sports_filterd['Critic_Score'].notna()]['Critic_Score']

0        7.7
4        8.0
16       7.9
17       8.0
68       8.3
        ... 
50561    5.0
50730    9.2
50731    3.6
50840    4.0
51761    9.1
Name: Critic_Score, Length: 688, dtype: float64

In [4]:
from scipy import stats as st
alpha = 0.05
result = st.ttest_1samp(sports_filterd[sports_filterd['Critic_Score'].notna()]['Critic_Score'], 5, alternative='greater')

print(sports_filterd[sports_filterd['Critic_Score'].notna()]['Critic_Score'].mean())
print(result)

if result.pvalue < alpha: 
    print('Отвергаем нулевую гипотезу, средняя оценка больше 5')
else:
    print('Не отвергаем нулевую нулевую гипотезу, спортивные игры не нравятся критикам')

7.2914244186046515
Ttest_1sampResult(statistic=40.864428307278445, pvalue=2.250074879862119e-186)
Отвергаем нулевую гипотезу, средняя оценка больше 5


P-value больше уровня значимости, значит нет оснований отвергнуть нулевую гипотрезу. Также мы видим, что средняя оценка спортивным играм, выставленная критиками, равна 7.3

### Критикам нравятся больше игры на PC или на PS4?

#### Гипотезы

H0 = оценка критиков для PC равна PS4

H1 = оценки критиков различаются для PC и PS4

In [5]:
PC = data[data['Platform'] == 'PC']
PC[PC['Critic_Score'].notna()]['Critic_Score']
PS4 = data[data['Platform'] == 'PS4']
PS4[PS4['Critic_Score'].notna()]['Critic_Score']

20       9.7
45       9.8
50       8.0
68       8.3
76       8.9
        ... 
52999    8.0
53775    7.9
54286    9.0
54659    8.9
55653    8.0
Name: Critic_Score, Length: 109, dtype: float64

In [6]:
result_p = st.ttest_ind(PC[PC['Critic_Score'].notna()]['Critic_Score'], PS4[PS4['Critic_Score'].notna()]['Critic_Score'], equal_var=False)
print(result_p)

if (result_p.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики по-разному относятся к играм на PC и PS4')
else:
    print('Не отвергаем нулевую гипотезу, критики относятся одинаково к играм на PC и PS4')

Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем нулевую гипотезу, критики по-разному относятся к играм на PC и PS4


In [7]:
average = data[data['Platform'].isin(['PC','PS4'])]['Critic_Score'].mean()
average

7.582443531827515

#### Гипотезы

H0 = оценка критиков для PC <= 7.58

H1 = ооценка критиков для PC > 7.58

In [8]:
alpha = 0.05
result_pc = st.ttest_1samp(PC[PC['Critic_Score'].notna()]['Critic_Score'], average, alternative='greater')

print(PC[PC['Critic_Score'].notna()]['Critic_Score'].mean())
print(result_pc)

if result_pc.pvalue < alpha: 
    print(f'Отвергаем нулевую гипотезу, средняя оценка больше {round(average,2)}')
else:
    print('Не отвергаем нулевую нулевую гипотезу, игры на PC нравятся критикам меньше, чем на PS4')

7.541849710982659
Ttest_1sampResult(statistic=-0.8318751695685387, pvalue=0.7971453690866915)
Не отвергаем нулевую нулевую гипотезу, игры на PC нравятся критикам меньше, чем на PS4


#### Гипотезы

H0 = оценка критиков для PS4C <= 7.58

H1 = ооценка критиков для PS4 > 7.58

In [9]:
alpha = 0.05
result_ps = st.ttest_1samp(PS4[PS4['Critic_Score'].notna()]['Critic_Score'], average, alternative='greater')

print(PS4[PS4['Critic_Score'].notna()]['Critic_Score'].mean())
print(result_ps)

if result_ps.pvalue < alpha: 
    print(f'Отвергаем нулевую гипотезу, средняя оценка больше {round(average,2)}. Критикам больше нравится PS4')
else:
    print('Не отвергаем нулевую нулевую гипотезу, игры на PS4 не нравятся критикам')

7.904587155963303
Ttest_1sampResult(statistic=2.6170773150717284, pvalue=0.00506990291872039)
Отвергаем нулевую гипотезу, средняя оценка больше 7.58. Критикам больше нравится PS4


* Критикам больше нравится PS4

### Критикам больше нравятся стрелялки или стратегии

H0 = оценка критиков для Shooter равна Strategy

H1 = оценки критиков различаются для Shooter и Strategy

In [11]:
Shooter = data[data['Genre'] == 'Shooter']
Shooter[Shooter['Critic_Score'].notna()]['Critic_Score']
Strategy = data[data['Genre'] == 'Strategy']
Strategy[Strategy['Critic_Score'].notna()]['Critic_Score']
average_g = data[data['Genre'].isin(['Shooter','Strategy'])]['Critic_Score'].mean()
average_g

7.330198019801981

In [12]:
result_g = st.ttest_ind(Shooter[Shooter['Critic_Score'].notna()]['Critic_Score'], Strategy[Strategy['Critic_Score'].notna()]['Critic_Score'], equal_var=False)

print(result_g)

print('Shooter', round(Shooter[Shooter['Critic_Score'].notna()]['Critic_Score'].mean(),1))
print('Strategy', round(Strategy[Strategy['Critic_Score'].notna()]['Critic_Score'].mean(),1))
if (result_g.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики по-разному относятся к играм на PC и PS4')
else:
    print('Не отвергаем нулевую гипотезу, критики относятся к шутерам и стратегиям одинаково')

Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Shooter 7.3
Strategy 7.4
Не отвергаем нулевую гипотезу, критики относятся к шутерам и стратегиям одинаково


* Критики примерно одинаково оценивают стрелялки и стратегии

## Задание 2

In [3]:
df = pd.read_csv('C:\\Users\\Irina\\Documents\\Материалы PYDA\\Stats\\case_study\\spam.txt')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# приведем все слова к нижнему регистру
df['message'] = df['Message'].str.lower()
df.head()

,Category,Message,message
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro..."


In [25]:
df.head()

,Category,Message,message,only_words_text,splitted,wo_stop_words,lemmatized,lemmatized_str
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...",go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazy, available, bugis, n...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,ok lar joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]",ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin...",free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,u dun say so early hor u c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, early, hor, u, c, already, say]",u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...",nah i don t think he goes to usf he lives arou...,"[nah, i, don, t, think, he, goes, to, usf, he,...","[nah, think, goes, usf, lives, around, though]","[nah, think, go, usf, life, around, though]",nah think go usf life around though


In [9]:
#remove symbols
df['only_words_text'] = df['message'].str.replace(r'[\W_]+',' ', regex=True)

In [10]:
# split by words
df['splitted'] = df['only_words_text'].str.split()

In [12]:
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))
df['wo_stop_words'] = df['splitted'].apply(lambda x: [item for item in x if item not in stopwords_set])

In [14]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
df['lemmatized'] = [[wordnet_lemmatizer.lemmatize(word) for word in text] for text in df['wo_stop_words']]

In [24]:
df['lemmatized_str'] = df.lemmatized.apply(lambda x: ' '.join([str(i) for i in x]))

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.lemmatized_str)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
from sklearn.model_selection import train_test_split

X = tfidf_matrix
y= df.Category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [37]:
X_train.shape

(3900, 8048)

In [38]:
y.shape

(5572,)

In [45]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42).fit(X_train, y_train)
predict = clf.predict(X_test)
predict
clf.predict_proba(X_test)


array([[0.93413638, 0.06586362],
       [0.8806768 , 0.1193232 ],
       [0.96440101, 0.03559899],
       ...,
       [0.96572208, 0.03427792],
       [0.41238101, 0.58761899],
       [0.96496257, 0.03503743]])

In [47]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, predict)
confusion

array([[1445,    3],
       [  66,  158]], dtype=int64)

### 1445 и 158 значений были правильно спрогнозированы моделью и 69 неправильно

In [48]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predict, target_names=["ham", "spam"]))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1448
        spam       0.98      0.71      0.82       224

    accuracy                           0.96      1672
   macro avg       0.97      0.85      0.90      1672
weighted avg       0.96      0.96      0.96      1672



In [56]:
result = pd.DataFrame()
result['y_test'] = y_test
result['predict'] = predict

In [60]:
result['check'] = result['y_test'] == result['predict']


In [66]:
final = result.join(df['Message'], how ='left')
final

,y_test,predict,check,Message
3245,ham,ham,True,Squeeeeeze!! This is christmas hug.. If u lik ...
944,ham,ham,True,And also I've sorta blown him off a couple tim...
1044,ham,ham,True,Mmm thats better now i got a roast down me! i...
2484,ham,ham,True,Mm have some kanji dont eat anything heavy ok
812,ham,ham,True,So there's a ring that comes with the guys cos...
...,...,...,...,...
2505,ham,ham,True,"Hello, my boytoy! I made it home and my consta..."
2525,spam,spam,True,FREE entry into our £250 weekly comp just send...
4975,ham,ham,True,Aiyo u so poor thing... Then u dun wan 2 eat? ...
650,spam,spam,True,"You have won ?1,000 cash or a ?2,000 prize! To..."


In [67]:
final[final['check'] == False]

,y_test,predict,check,Message
2952,ham,spam,False,Hey now am free you can call me.
881,spam,ham,False,Reminder: You have not downloaded the content ...
1961,spam,ham,False,Guess what! Somebody you know secretly fancies...
3864,spam,ham,False,Oh my god! I've found your number again! I'm s...
2575,spam,ham,False,Your next amazing xxx PICSFREE1 video will be ...
...,...,...,...,...
4543,spam,ham,False,FreeMsg Hi baby wow just got a new cam moby. W...
752,spam,ham,False,You have an important customer service announc...
309,spam,ham,False,TheMob> Check out our newest selection of cont...
495,ham,spam,False,Are you free now?can i call now?
